### 抓取163博客日志

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月14日

----------

- 研究目的：获取163博客某个博主的日志；
- 研究方法：从163博客网站爬取博主所有日志。

In [3]:
#引入包
import pandas as pd
import requests,re,time

# 自定义
import sys
sys.path.append('/home/jquser')

from dm import *
from tl import exists_file


# 数据目录
_data_path='../../Data/'
# 数据引擎
_data=Csv(_data_path)



# 163博客返回的为dwr数据格式
# 必须采用headers请求
# 配置param
# requests.post(url,data=param,headers=headers)

# 页面请求头
headers={ 
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.8',
    'Connection':'keep-alive',
    'Content-Length':'274',
    'Content-Type':'text/plain',
    'Host':'api.blog.163.com',
    'Origin':'http://api.blog.163.com',
    'Referer':'http://api.blog.163.com/crossdomain.html?t=20100205',
    'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 \
    Safari/537.36 Core/1.53.4620.400 QQBrowser/9.7.13014.400}  ',
}
   
# 请求参数
param={ 
    'callCount':'1',
    'scriptSessionId':'${scriptSessionId}187',
    'c0-scriptName':'BlogBeanNew',
    'c0-methodName':'getBlogs',
    'c0-id':'0',
    'c0-param0':'number:60278066',#用户id
    'c0-param1':'number:0',#第几条
    'c0-param2':'number:1',#一次几条
    'batchId':'318074',
    }

# 返回的数据格式

# //#DWR-REPLY
# var s0={};var s1={};var s2=[];s0.abstractSysGen=1;s0.accessCount=5;s0.allowComment=-100;s0.allowView=-100;
# s0.blogAbstract="<P>\u76F8\u805A\u4E0D\u77E5\u6101\u6ECB\u5473\uFF0C</P>\r\n<P>\u6B22\u7B11\u65E0\u5FE7\uFF0C</P>\r\n<P>
# \u6B22\u7B11\u65E0\u5FE7\uFF0C</P>\r\n<P>\u6E38\u620F\u7EC8\u65E5\u5F3A\u8BF4\u6101\u3002</P>\r\n<P>&nbsp;</P>\r\n<P>
# \u522B\u540E\u65B9\u77E5\u6101\u6ECB\u5473\uFF0C</P>\r\n<P>\u6B32\u8BF4\u8FD8\u4F11\uFF0C</P>\r\n<P>
# \u6B32\u8BF4\u8FD8\u4F11\uFF0C</P>\r\n<P>\u5374\u9053\u6625\u522B\u805A\u65F6\u79CB\u3002</P>";
# s0.blogAttachments=null;s0.blogCount=s1;s0.blogExt=null;s0.circleCount=0;s0.circleIdList=s2;s0.circleIds=null;
# s0.classId="fks_087064093095089071093087082095085087080071082081085067085";s0.className="C.\u8BCD";s0.commentCount=0;
# s0.comments=null;s0.content="<P>\u76F8\u805A\u4E0D\u77E5\u6101\u6ECB\u5473\uFF0C</P>\r\n<P>\u6B22\u7B11\u65E0\u5FE7\uFF0C
# </P>\r\n<P>\u6B22\u7B11\u65E0\u5FE7\uFF0C</P>\r\n<P>\u6E38\u620F\u7EC8\u65E5\u5F3A\u8BF4\u6101\u3002</P>\r\n<P>&nbsp;
# </P>\r\n<P>\u522B\u540E\u65B9\u77E5\u6101\u6ECB\u5473\uFF0C</P>\r\n<P>\u6B32\u8BF4\u8FD8\u4F11\uFF0C</P>\r\n<P>
# \u6B32\u8BF4\u8FD8\u4F11\uFF0C</P>\r\n<P>\u5374\u9053\u6625\u522B\u805A\u65F6\u79CB\u3002</P>";
# s0.contentPlainText=null;s0.id="fks_082067085083086066084094094095085087080071082081085067085";s0.ip="221.130.168.7";
# s0.isBlogAbstractComplete=true;s0.isPublished=1;s0.keyName="ID";s0.keyWordCheckedState=0;
# s0.lastAccessCountUpdateTime=1523684498270;s0.matchedKeyWord=false;s0.modifyTime=0;s0.moveFrom="NONE";
# s0.permaSerial="1114673012009225115942319";s0.permalink="blog/static/1114673012009225115942319";s0.photoIds=null;
# s0.photoStoreTypes=null;s0.publishTime=1237953582319;s0.publishTimeStr="11:59:42";s0.publisherId=0;
# s0.publisherNickname=null;s0.publisherUsername=null;s0.rank=0;s0.recomBlogHome=false;s0.ref=false;
# s0.shortPublishDateStr="2009-3-25";s0.synchLofter=-1;s0.synchMiniBlog=-1;s0.tag="";s0.title="\u91C7\u6851\u5B50";
# s0.trackbackCount=0;s0.trackbackUrl="blog/1114673012009225115942319.track";s0.userId=111467301;s0.userName="dgczy@126";
# s0.userNickname="dgczy";s0.valid=0;s0.zipContent=null;
# s1.accessCount=5;s1.blogId=401571088;s1.commentCount=0;s1.likeCount=0;s1.mainCommentCount=0;
# s1.permaSerial="1114673012009225115942319";s1.postHot=0;s1.recommendCount=0;s1.trackbackCount=0;s1.userId=111467301;
# dwr.engine._remoteHandleCallback('318074','0',[s0]);


# 获取微博发帖
def get_blog_data(user_name,user_id,count,begin=0):
    #请求链接
    url='http://api.blog.163.com/%s/dwr/call/plaincall/BlogBeanNew.getBlogs.dwr'%(user_name)
    #配置用户id
    param['c0-param0']='number:%s'%(user_id)
    #数据结构
    df=pd.DataFrame()
    data_list=[]
    columns=['title','publishdate','permalink','content','sended']
    #遍历所有发帖   
    for i in range(begin,count):
        #配置每次获取日志的序号
        param['c0-param1']='number:%s'%str(i)
        #请求数据
        r=requests.post(url,data=param,headers=headers)
        #返回失败，退出循环
        if (r.ok is False) :
            break
        r=r.text 
        #unicode转换为中文
        r=r.encode('utf8').decode('unicode_escape') 
        print(re.split("""(?<!\"[^;]+);(?![^;]+\")""", r))
        1/0
        #解析标题
        title=r.split('s0.title="')[1].split('";')[0]
        #解析发布日期
        publishdate=r.split('s0.shortPublishDateStr="')[1].split('";')[0]
        #解析发布时间
        publishtime=r.split('s0.publishTimeStr="')[1].split('";')[0]
        #静态网页网址
        permalink=r.split('s0.permalink="')[1].split('";')[0] 
        #正文
        content=r.split('s0.content="')[1].split('";')[0] 
        #添加信息到数据表
        data_list.append([title,'%s %s'%(publishdate,publishtime),permalink,content,False])
        #延时5秒，防止被踢
#         time.sleep(2)
        print(' 已获取：第 %s 条，%s，%s %s'%(str(i),title,publishdate,publishtime),end='\r')
    #生成数据表    
    df=pd.DataFrame(data_list,columns=columns)   
    #返回数据
    return df


# 博主信息网址格式
# http://blog.163.com/dgczy@126/blog/#m=0

# 网页中博主信息
# UD.host = {
#       userId:111467301
#      ,userName:'dgczy@126'
#      ,nickName:'dgczy'
#      ,imageUpdateTime:-1
#      ,baseUrl:'http://blog.163.com/dgczy@126/'
#      ,gender:'他'
#      ,email:'dgczy@126.com'
#      ,photo163Name:'dgczy@126'
#      ,photo163HostName:'dgczy@126'
#      ,TOKEN_HTMLMODULE:''
#      ,isMultiUserBlog:false
#      ,isWumiUser:true
#      ,sRank:-100
#   };

# 网页中分类信息，计算count后的数字，得到所有日志数量
# c:[{id:'-2',name:'草稿箱',count:0},{id:'-3',name:'回收站',count:0},
# {id:"fks_087071085083088066093082087095085087080071082081085067085",name:"A.日记",count:4},
# {id:"fks_087064093095089071093086083095085087080071082081085067085",name:"B.家",count:4},
# {id:"fks_087064093095089071093087082095085087080071082081085067085",name:"C.词",count:20},
# {id:"fks_087064093095089071093087085095085087080071082081085067085",name:"D.诗歌集",count:0},
# {id:"fks_087064093095089071093086095095085087080071082081085067085",name:"E.密码学初探",count:1},
# {id:"fks_087064093095089071093086080095085087080071082081085067085",name:"F.阿芒思选",count:1},
# {id:"fks_087071086087083070080086087095085087080071082081085067085",name:"G.篆刻",count:1},
# {id:"fks_087070085084083069081083083095085087080071082081085067085",name:"H.设计作品",count:0},
# {id:"fks_087071086086081065085086080095085087080071082081085067085",name:"H.文章转载",count:2},
# {id:"fks_087064093095088067087081087095085087080071082081085067085",name:"默认分类",count:0}],

#获取博主信息
def get_blog_userinfo(user_name):
    #所有帖子链接（该网页含有用户信息及帖子数）    
    url='http://blog.163.com/%s/blog/#m=0'%(user_name)
    #请求数据
    ret=requests.get(url).text
    #获取用户信息
    r=ret.split('UD.host = {')[1].split('};')[0]
    r=r.replace("'","").replace(" ","").replace("\r\n","")
    #解析用户信息
    user_id=r.split('userId:')[1].split(',')[0]
    user_name=r.split('userName:')[1].split(',')[0]
    user_nickname=r.split('nickName:')[1].split(',')[0]
    user_baseurl=r.split('baseUrl:')[1].split(',')[0]
    user_email=r.split('email:')[1].split(',')[0]
    user_gender=r.split('gender:')[1].split(',')[0]
    user_gender='男' if user_gender=='他' else '女' 
    #解析帖子数量
    r=ret.split('c:[')[1].split('],')[0]
    r=re.findall(r"count:(.*?)}",r,re.S)
    user_count=0
    for i in r[2:]:
        user_count+=int(i)
    return (user_id,user_name,user_nickname,user_baseurl,user_email,user_gender,user_count)


#读取数据        
def read_blog_data(user_id):
    #文件名
    data_file='msg_blog_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    return df

#保存数据        
def save_blog_data(user_id,df):
    #文件名
    data_file='msg_blog_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)


#博客博主注册名
user_list=[
    'gaoya',
    ]

print(' 已准备好!') 

 已准备好!


In [41]:
df=get_blog_data(user_name,user_id,user_count,begin=0)



error: look-behind requires fixed-width pattern

## 1.博主信息

In [4]:
# 获取博主信息
(user_id,user_name,user_nickname,user_baseurl,user_email,user_gender,user_count)=get_blog_userinfo(user_list[0])
print(' ID　　 ',user_id)
print(' 注册名 ',user_name)
print(' 昵称　 ',user_nickname)
print(' 网址　 ',user_baseurl)
print(' 邮箱　 ',user_email)
print(' 性别　 ',user_gender)
print(' 日志数 ',user_count)

 ID　　  60278066
 注册名  gaoya
 昵称　  Peter
 网址　  http://gaoya.blog.163.com/
 邮箱　  gaoya@163.com
 性别　  男
 日志数  505


## 2.爬取数据
- 采用断点续爬。

In [5]:
#断点续爬数据
def update_blog_data(user_name,user_id,user_count):
    #检查是否已存在数据
    try:
        #已存在数据
        df=read_blog_data(user_id)
        #起始位置为当前记录数
        new_df=get_blog_data(user_name,user_id,user_count,begin=len(df)) 
        if len(new_df)==0:
            print('没有新帖子或更新不成功！')
        else:
            #拼接新旧数据
            df=pd.concat([df,new_df],ignore_index=True)
            #保存到cvs文件
            save_blog_data(user_id,df) 
            print(' 共更新：%s 条帖子'%len(new_df))    
    except:
        #数据不存在，起始位置为0 
        df=get_blog_data(user_name,user_id,user_count,begin=0) 
        if len(df)==0:
            print('获取不成功！')
        else:
            #保存到cvs文件
            save_blog_data(user_id,df) 
            print(' 共获取：%s 条帖子'%len(df))
    
update_blog_data(user_name,user_id,user_count)

没有新帖子或更新不成功！


## 3.发送邮件

In [17]:
import os

#引入自定义（jsl）库
#保存当前目录
path=os.getcwd()
#更改到用户根目录为
os.chdir("/home/jquser/")
#导入自己的库
from mail import *
#恢复当前目录
os.chdir(path)

#发送邮件
def send_blog_data(user_id):
    #读取数据
    df=read_blog_data(user_id)
    issend=False
    for i in range(0,len(df)):
        if df['sended'].iloc[i]==True:
            continue
        #邮件主题
        subjiect='%s（博客日志 %s）'%(df.title.iloc[i],user_name.upper())
        #邮件正文
        message='<h2>%s</h2><p>%s</br>%s</p><hr /></br>%s'%(df.title.iloc[i],df.publishdate.iloc[i],
                                               user_baseurl+df.permalink.iloc[i],df.content.iloc[i])
        try:
            #发送邮件
            send_html_qqmail(subjiect,message)
            #发送成功标志
            df['sended'].iloc[i]=True
            issend=True
            #延时5秒，防止被踢
            time.sleep(5)
            print(' 已发送：%s'%(subjiect),end='\r')
        except Exception as e:
            print(' 错误：%s'%e)
            break
    if issend==True:
        #保存到cvs文件
        save_blog_data(user_id,df) 
        print(' 发送完毕！')
    else:
        print(' 没有要发送的！')
        
send_blog_data(user_id)     

 发送完毕！


/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## 4.产看数据表

In [6]:
df=read_blog_data(user_id)
# df['sended']=False
# save_blog_data(user_id,df)
df

,title,publishdate,permalink,content,sended
0,再说资产配置,2014-3-23 21:43:47,blog/static/6027806620142239384860,刚才看到有博友问红利etf是否值得投资。那我就总体说说怎么根据资产配置原则选择etf。<di...,True
1,各种量化投资计划电梯,2014-1-4 13:14:14,blog/static/6027806620140405224549,在资本市场，成功的投资一般三种方式：<wbr><div><br></div><div>价值投...,True
2,送给新朋友的几句话,2016-1-26 10:47:10,blog/static/602780662016026104710643,最近雪球、微博的观光团朋友们纷纷来看这个记录了本人几年投资历程的博客。<wbr><div><...,True
3,油气网格开始,2015-7-15 9:34:32,blog/static/60278066201561593432546,分级A回款到账，现在有钱又有闲。开始搞油气。<wbr><div><br></div><div...,True
4,很多东西不在这里写了。朋友可以去雪球关注,2015-7-10 10:03:26,blog/static/60278066201561010326313,etf拯救世界<wbr>,True
5,还有没有A类亏损的朋友了？,2015-6-25 14:45:23,blog/static/60278066201552524523151,站出来，我保证不打你。<wbr><div><br></div><div><img title...,True
6,大家误会了,2015-6-24 10:33:56,blog/static/602780662015524102551795,我不是说评论里谁说什么了我不高兴之类的。。大家还不了解我么，我还不了解大家么，而且，我有那么...,True
7,看了上一贴的评论，感觉是万万没想到,2015-6-23 14:11:00,blog/static/6027806620155232110140,以后在这里说品种，大概有两个选择：<wbr><div><br></div><div>第一，少...,True
8,同学们，来来来，都冒个泡,2015-6-19 13:43:04,blog/static/6027806620155191434417,在4500点，上证跌了600点之后，本博主在A类赚了一笔。资产增加得很愉快。<wbr><di...,True
9,欢迎添加公众号,2015-6-10 20:50:37,blog/static/60278066201551085037391,欢迎添加本博公众号，及时接收最新文章<div><br></div><div>微信扫一扫</d...,True
